In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
import future 
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
seed = 100

In [2]:
def accuracy(x,y):
    acc = 0
    for i in range(len(x)):
        if x[i,]==y[i,]:
            acc+=1
        else:
            acc+=0
    rate = acc/len(x)
    print('Accuracy rate is: ',rate)
    
    
def plot(image,ans):
    if len(image) > 10 | len(ans) > 10:
        print('TOO MANY PHOTOS IN ONE TIME!!')
        print('The maximum is 10 photos')
    for i in range(len(image)):
        plt.subplot(2,5,i+1)
        plt.imshow(image[i,...])
        plt.title(ans[i])

In [3]:
def importimages(path):
    folder_list =[]
    folder = glob.glob(path)
    for folder in folder :
        for name in glob.glob(folder+'/*'):
            folder_list.append(name)
            
    image_name =[]
    for folder in folder_list:
        for name in glob.glob(folder+'/*.jpg') :
            image_name.append(name)
            
    read_images = []
    for image in image_name:
        read_images.append(cv2.imread(image,cv2.IMREAD_GRAYSCALE))
    read_images = np.array(read_images)
    print ('Images are all in read_images')

In [4]:
path=r'C:\Users\\jim19\\Desktop\\picture'
folder_list =[]
folder = glob.glob(path)
for folder in folder :
    for name in glob.glob(folder+'/*'):
        folder_list.append(name)

In [5]:
image_name =[]
for folder in folder_list:
    for name in glob.glob(folder+'/*.jpg') :
        image_name.append(name)
            

In [6]:
image_show = dict()
read_images = []
i = 0
for image in image_name:
    img = cv2.imread(image)
    image_show[i] = img
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    read_images.append(img)
    i+=1
read_images = np.array(read_images)
print ('Images are all in read_images')

Images are all in read_images


In [7]:
print(read_images.shape)

(40000, 100, 100)


In [7]:
x = read_images
#split = np.random.choice(40000,8000,replace=False)

y = np.array(np.repeat(range(1,9),5000).reshape(-1,1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=seed,shuffle=True) 
x_test_image = x_test
x_test = x_test.reshape(x_test.shape[0],100,100,1)
x_train = x_train.reshape(x_train.shape[0],100,100,1)

print('Shape of x_train is (%d,%d,%d,%d)'%(x_train.shape))
print('Shape of x_test is (%d,%d,%d,%d)'%(x_test.shape))
print('Shape of y_train is (%d,%d)'%(y_train.shape))
print('Shape of y_test is (%d,%d)'%(y_test.shape))
onehot = OneHotEncoder()
y_train = onehot.fit_transform(y_train).toarray()
y_test = onehot.fit_transform(y_test).toarray()
print('Shape of y_train is (%d,%d)'%(y_train.shape))

Shape of x_train is (32000,100,100,1)
Shape of x_test is (8000,100,100,1)
Shape of y_train is (32000,1)
Shape of y_test is (8000,1)
Shape of y_train is (32000,8)


C:\Users\jim19\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\jim19\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

In [51]:
x_test.shape[0]

32000

In [8]:
# 淺層cnn
model=Sequential()
model.add(Flatten())
model.add(Dense(8,activation='softmax'))
#超過兩個就要選categorical_crossentrophy
tensorboard = TensorBoard(log_dir= 'logs\{}'.format('Tf'),histogram_freq=1)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# ##Check points
# filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=False,
# mode='max')


model.fit(x_train,y_train,batch_size = 100,validation_split=0.2,epochs = 3)


Train on 25600 samples, validate on 6400 samples
Epoch 1/3
25600/25600 [==============================] - 4s 142us/sample - loss: 287.0252 - accuracy: 0.2259 - val_loss: 201.4348 - val_accuracy: 0.2295
Epoch 2/3
25600/25600 [==============================] - 3s 109us/sample - loss: 248.4656 - accuracy: 0.2521 - val_loss: 354.7506 - val_accuracy: 0.2502
Epoch 3/3
25600/25600 [==============================] - 3s 107us/sample - loss: 262.0520 - accuracy: 0.2552 - val_loss: 221.6296 - val_accuracy: 0.2400


In [13]:

category_ = {0:'Mosaic', 1:'Shrink', 2:'Expansion',3: 'Brightness',4: 'Contrast', 5: 'Sharpen', 6:'Paste', 7:'Original'}
def wheremax(list_,n=8):
    m = max (list_)
    for i in range(len(list_)):
        if list_[i] == m:
            where = i
    which_how = [where,m]
    return which_how
print('修圖為{}的機率為{}'.format(category_[wheremax(model.predict(x_test)[2])[0]],wheremax(model.predict(x_test)[2])[1]))

修圖為Sharpen的機率為1.0


In [13]:
def tag_on_images(image_show = image_show):
    image_all = dict()
    for i in range(len(image_show)):
        img = image_show[i]
        text = '修圖為{}的機率為{}'.format(category_[wheremax(model.predict(x_test)[i])[0]],wheremax(model.predict(x_test)[i])[1])
        cv2.putText(img,text, (0,100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 1)
        image_all[i] = img

In [31]:
i = 31
img = image_show[i]
text = '{},{}%'.format(category_[wheremax(model.predict(x_test)[i])[0]],round(wheremax(model.predict(x_test)[i])[1])*100,1)
cv2.putText(img,text, (5,100), cv2.FONT_HERSHEY_COMPLEX, 0.3, (0, 0, 0), 1)
cv2.imshow('0',img)
k = cv2.waitKey(0) # waitkey代表读取键盘的输入，括号里的数字代表等待多长时间，单位ms。 0代表一直等待
if k ==27:     # 键盘上Esc键的键值
    cv2.destroyAllWindows() 
print(text)

Sharpen,100.0%


In [30]:
img1 = image_show[1]
cv2.putText(img1,' 80 %', (0,100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 1)

def wheremax(list_,n=8):
    m = max (list_)
    where = [i for i in range(n) if m == list_[i]]
    return where
wheremax(model.predict(x_test)[1])
cv2.imshow('0',img1)
k = cv2.waitKey(0) # waitkey代表读取键盘的输入，括号里的数字代表等待多长时间，单位ms。 0代表一直等待
if k ==27:     # 键盘上Esc键的键值
    cv2.destroyAllWindows() 

In [61]:
from tensorflow.keras.layers import Input
input_tensor = Input(shape=(100, 100, 1))

In [68]:
base_model = Sequential()
base_model.add(InceptionV3(input_tensor=input_tensor,weights=None, include_top=False))
base_model.add(Flatten())
base_model.add(Dense(8,activation = 'softmax'))

In [67]:
base_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 1, 1, 2048)        21802208  
_________________________________________________________________
dense_5 (Dense)              (None, 1, 1, 8)           16392     
Total params: 21,818,600
Trainable params: 21,784,168
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
base_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
train_history = base_model.fit(x=x_train, y=y_train, validation_split=0.3, epochs=2, batch_size=80, verbose=2)  

Train on 22400 samples, validate on 9600 samples
Epoch 1/2
22400/22400 - 2612s - loss: 1.5252 - accuracy: 0.3822 - val_loss: 2.0814 - val_accuracy: 0.2642
Epoch 2/2


In [13]:
model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=10)

 16801792/219055592 [=>............................] - ETA: 23:47

KeyboardInterrupt: 

In [9]:
model=Sequential()

model.add(Flatten())
model.add(Dense(8,activation='relu'))
model.add(Flatten())
model.add(Dense(8,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
train_history = model.fit(x=x_train, y=y_train, epochs=2, batch_size=80, verbose=2)  

Train on 32000 samples
Epoch 1/2
32000/32000 - 4s - loss: 2.3417 - accuracy: 0.1229
Epoch 2/2
32000/32000 - 4s - loss: 2.0796 - accuracy: 0.1226


In [12]:
prediction = np.argmax(model.predict(x_test),axis=1)

In [31]:
prediction[3000:3500]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [2]:
data_size = 1000
# 80% of the data is for training.
train_pct = 0.8

train_size = int(data_size * train_pct)

# Create some input data between -1 and 1 and randomize it.
x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

# Generate the output data.
# y = 0.5x + 2 + noise
y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

# Split into test and train pairs.
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

In [16]:

log_dir= 'logs\{}'.format('Tiamo')
tensorboard = TensorBoard(log_dir,histogram_freq=1)

model = Sequential([
    Dense(16, input_dim=1),
    Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=SGD(lr=0.2),
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=0, # Suppress chatty output; use Tensorboard instead
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard]
)

print("Average test loss: ", np.average(training_history.history['loss']))

Training ... With default parameters, this takes less than 10 seconds.
Average test loss:  0.043242648539599034


In [11]:
%tensorboard --logdir

UsageError: Line magic function `%tensorboard` not found.


In [15]:
%tensorboard --logdir logs/scalars

UsageError: Line magic function `%tensorboard` not found.
